<a href="https://colab.research.google.com/github/zamalex/BakingFinal/blob/master/TRANSFORMERS_UCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Uncomment if running on googlecolab
!pip install hickle
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/PerCom2021-FL-master/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 1.8 MB/s eta 0:00:00
Mounted at /content/drive/
[Errno 2] No such file or directory: 'drive/MyDrive/PerCom2021-FL-master/'
/content


In [ ]:
import numpy as np
import os
import pandas as pd
from subprocess import call
import requests
np.random.seed(0)
import urllib.request
import zipfile
import hickle as hkl
import tensorflow as tf

In [ ]:
# functions for loading and downloading the dataset

# load a single file as a numpy array
def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files, such as x, y, z data for a given variable
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = np.dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	filenames = list()
	# body acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# Framing data by windows
def segmentData(accData,time_step,step):
    segmentAccData = list()
    for i in range(0, accData.shape[0] - time_step,step):
        segmentAccData.append(accData[i:i+time_step,:])
    return segmentAccData

# download function for datasets
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [ ]:
# download and unzipping dataset
os.makedirs('dataset',exist_ok=True)
print("downloading...")
data_directory = os.path.abspath("dataset/UCI HAR Dataset.zip")
if not os.path.exists(data_directory):
    download_url("https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI HAR Dataset.zip",data_directory)
    print("download done")
else:
    print("dataset already downloaded")

data_directory2 = os.path.abspath("dataset/UCI HAR Dataset")
if not os.path.exists(data_directory2):
    print("extracting data")
    with zipfile.ZipFile(data_directory, 'r') as zip_ref:
        zip_ref.extractall(os.path.abspath("dataset/"))
    print("data extracted in " + data_directory2)
else:
    print("Data already extracted in " + data_directory2)

downloading...
download done
extracting data
data extracted in /content/dataset/UCI HAR Dataset


In [ ]:
trainSubjectList = pd.read_csv('dataset/UCI HAR Dataset/train/subject_train.txt', header=None, delim_whitespace=True).values
testSubjectList = pd.read_csv('dataset/UCI HAR Dataset/test/subject_test.txt', header=None, delim_whitespace=True).values

In [ ]:
# load all train
trainX, trainy = load_dataset('train', 'dataset/UCI HAR Dataset/')
trainy = np.asarray([x - 1 for x in trainy])

# load all test
testX, testy = load_dataset('test', 'dataset/UCI HAR Dataset/')
testy = np.asarray([x - 1 for x in testy])

In [ ]:
combinedData = np.vstack((trainX,testX))

In [ ]:
subjectList = np.vstack((trainSubjectList,testSubjectList)).squeeze()
labels = np.vstack((trainy,testy)).squeeze()

In [ ]:
nbOfSubjects = len(np.unique(subjectList))

In [ ]:
subjectDataDict = {new_list: [] for new_list in range(nbOfSubjects)}
subjectLabelDict = {new_list: [] for new_list in range(nbOfSubjects)}

In [ ]:
meanAcc = np.mean(combinedData[:,:,:3])
stdAcc = np.std(combinedData[:,:,:3])
varAcc = np.var(combinedData[:,:,:3])

meanGyro = np.mean(combinedData[:,:,3:])
stdGyro = np.std(combinedData[:,:,3:])
varGyro = np.var(combinedData[:,:,3:])

normalizedAllAcc = (combinedData[:,:,:3] - meanAcc) / stdAcc
normalizedAllGyro = (combinedData[:,:,3:] - meanGyro) / stdGyro
normalizedAll = np.dstack((normalizedAllAcc,normalizedAllGyro))

In [ ]:
trainy = np.squeeze(trainy)
testy = np.squeeze(testy)

In [ ]:
dataName = 'UCI'
os.makedirs('datasetStandardized/'+dataName, exist_ok=True)
hkl.dump(normalizedAll[:trainX.shape[0]],'datasetStandardized/'+dataName+ '/trainX.hkl' )
hkl.dump(normalizedAll[trainX.shape[0]:],'datasetStandardized/'+dataName+ '/testX.hkl' )
hkl.dump(trainy,'datasetStandardized/'+dataName+ '/trainY.hkl' )
hkl.dump(testy,'datasetStandardized/'+dataName+ '/testY.hkl' )

In [26]:
trainy.shape

(7352,)

In [17]:
!pip install tensorflow==2.10.1


In [24]:
!pip install -r requirements.txt


  Using cached cachetools-5.3.0-py3-none-any.whl (9.3 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached charset_normalizer-3.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (198 kB)
  Using cached flatbuffers-23.1.21-py2.py3-none-any.whl (26 kB)
  Using cached fonttools-4.38.0-py3-none-any.whl (965 kB)
  Using cached google_auth-2.16.0-py2.py3-none-any.whl (177 kB)
  Using cached grpcio-1.51.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached importlib_metadata-6.0.0-py3-none-any.whl (21 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached keras_flops-0.1.2-py3-none-any.whl (5.3 kB)
  Using cached libclang-15.0.6.1-py2.py3-none-manylinux2010_x86_64.whl (21.5 MB)
  Using cached Markdown-3.4.1-py3-none-any.whl (93 kB)
  Using cached MarkupSafe-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Using cached matplotlib-3.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014

In [22]:
%ls

dataset/  datasetStandardized/  drive/  sample_data/


In [32]:
!pip install keras_flops

  Using cached keras_flops-0.1.2-py3-none-any.whl (5.3 kB)


In [33]:
import os
randomSeed = 1
os.environ['PYTHONHASHSEED']=str(randomSeed)
import numpy as np
import tensorflow as tf
import csv
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import time
import hickle as hkl
import random
import math
import logging
import shutil
import gc
import sys
import sklearn.manifold
import seaborn as sns
import argparse
import matplotlib.gridspec as gridspec
import __main__ as main
import model
import utils
from keras_flops import get_flops

In [34]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [35]:
# which CPU/GPU to use
# "-1,0,1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Set The Default Hyperparameters Here

architecture = "HART"
# MobileHART, HART

# RealWorld,HHAR,UCI,SHL,MotionSense, COMBINED
dataSetName = 'UCI'

#BALANCED, UNBALANCED
dataConfig = "BALANCED"

# Show training verbose: 0,1
showTrainVerbose = 1

# input window size
segment_size = 128

# input channel count
num_input_channels = 6

learningRate = 3e-4

# model drop out rate
dropout_rate = 0.3

# local epoch
localEpoch = 200
# or 4
frameLength = 16

timeStep = 16

positionDevice = ''
# ['chest','forearm','head','shin','thigh','upperarm','waist']
# ['nexus4', 'lgwatch','s3', 's3mini','gear','samsungold']
tokenBased = False

measureEnergy = False

In [36]:
# hyperparameter for the model

batch_size = 256
projection_dim = 192
filterAttentionHead = 4

# To adjust the number of blocks in for HART, Add or remove the conv kernel size here
# each conv kernel length is for one HART block
convKernels = [3, 7, 15, 31, 31, 31]

In [37]:
def add_fit_args(parser):
    """
    parser : argparse.ArgumentParser
    return a parser added with args required by fit
    """
    # Training settings
    parser.add_argument('--batch_size', type=int, default=batch_size,
                        help='Batch size of the training')
    parser.add_argument('--localEpoch', type=int, default=localEpoch,
                        help='Number of epochs for training')
    parser.add_argument('--architecture', type=str, default=architecture,
                        help='Choose between HART or MobileHART')
    parser.add_argument('--projection_dim', type=int, default=projection_dim,
                        help='Size of the projection dimensions')
    parser.add_argument('--frame_length', type=int, default=frameLength,
                help='Patch Size')
    parser.add_argument('--time_step', type=int, default=timeStep,
            help='Stride Size')
    parser.add_argument('--dataset', type=str, default=dataSetName,
        help='Dataset')
    parser.add_argument('--tokenBased', type=bool, default=tokenBased,
        help='Use Token or Global Average Pooling')
    parser.add_argument('--positionDevice', type=str, default=positionDevice,
        help='Test is done other position not in training, if empty, uses a 70/10/20 train/dev/test ratio ')
    args = parser.parse_args()
    return args

In [38]:
def is_interactive():
    return not hasattr(main, '__file__')

In [39]:
if not is_interactive():
    args = add_fit_args(argparse.ArgumentParser(description='Human Activity Recognition Transformer'))
    localEpoch = args.localEpoch
    batch_size = args.batch_size
    architecture = args.architecture
    projection_dim = args.projection_dim
    frameLength = args.frame_length
    timeStep = args.time_step
    dataSetName = args.dataset
    tokenBased = args.tokenBased
    positionDevice = args.positionDevice

input_shape = (segment_size,num_input_channels)
projectionHalf = projection_dim//2
projectionQuarter = projection_dim//4

transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers

R = projectionHalf // filterAttentionHead
assert R * filterAttentionHead == projectionHalf


segmentTime = [x for x in range(0,segment_size - frameLength + timeStep,timeStep)]
assert R * filterAttentionHead == projectionHalf
if(positionDevice != ''):
    assert dataSetName == "RealWorld" or dataSetName == "HHAR"

In [40]:
# specifying activities and where the results will be stored
if(dataSetName == 'UCI'):
    ACTIVITY_LABEL = ['Walking', 'Upstair','Downstair', 'Sitting', 'Standing', 'Lying']
elif(dataSetName == "RealWorld"):
    ACTIVITY_LABEL = ['Downstairs','Upstairs', 'Jumping','Lying', 'Running', 'Sitting', 'Standing', 'Walking']
elif(dataSetName == "Motion_Sense"):
    ACTIVITY_LABEL = ['Downstairs', 'Upstairs', 'Sitting', 'Standing', 'Walking', 'Jogging']
elif(dataSetName == "HHAR"):
    ACTIVITY_LABEL = ['Sitting', 'Standing', 'Walking', 'Upstair', 'Downstairs', 'Biking']
else:
#     SHL
    ACTIVITY_LABEL = ['Standing','Walking','Runing','Biking','Car','Bus','Train','Subway']

activityCount = len(ACTIVITY_LABEL)

architectureType = str(architecture)+'_'+str(int(frameLength))+'frameLength_'+str(timeStep)+'TimeStep_'+str(projection_dim)+"ProjectionSize_"+str(learningRate)+'LR'
if(tokenBased):
    architectureType = architectureType + "_tokenBased"

if(positionDevice != ''):
    architectureType = architectureType + "_PositionWise_" + str(positionDevice)
mainDir = './'

if(localEpoch < 20):
    architectureType =  "Tests/"+str(architectureType)
filepath = mainDir +'HART_Results/'+architectureType+'/'+dataSetName+'/'
os.makedirs(filepath, exist_ok=True)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

attentionPath = filepath+"attentionImages/"
os.makedirs(attentionPath, exist_ok=True)

bestModelPath = filepath + 'bestModels/'
os.makedirs(bestModelPath, exist_ok=True)

currentModelPath = filepath + 'currentModels/'
os.makedirs(currentModelPath, exist_ok=True)


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
np.random.seed(randomSeed)
tf.keras.utils.set_random_seed(randomSeed)
tf.random.set_seed(randomSeed)
random.seed(randomSeed)


Num GPUs Available:  1


In [42]:
if(dataSetName == "COMBINED"):
    datasetList = ["UCI","RealWorld","HHAR", "Motion_Sense","SHL_128"]
    ACTIVITY_LABEL = ['Walk', 'Upstair', 'Downstair', 'Sit', 'Stand', 'Lay', 'Jump','Run', 'Bike', 'Car', 'Bus', 'Train', 'Subway']
    activityCount = len(ACTIVITY_LABEL)
    UCI = [0,1,2,3,4,5]
    REALWORLD_CLIENT = [2,1,6,5,7,3,4,0]
    HHAR = [3,4,0,1,2,8]
    Motion_Sense = [2,1,3,4,0,7]
    SHL = [4,0,7,8,9,10,11,12]

    centralTrainData = []
    centralTrainLabel = []
    centralTestData = []
    centralTestLabel = []
    for dataSetName in datasetList:
        clientCount = utils.returnClientByDataset(dataSetName)
        loadedDataset = utils.loadDataset(dataSetName,clientCount,dataConfig,randomSeed,mainDir+'datasets/')
        centralTrainData.append(loadedDataset.centralTrainData)
        centralTrainLabel.append(loadedDataset.centralTrainLabel)
        centralTestData.append(loadedDataset.centralTestData)
        centralTestLabel.append(loadedDataset.centralTestLabel)
        print(dataSetName + " has class :" +str(np.unique(centralTrainLabel[-1])))
        del loadedDataset

    centralTestLabelAligned = []
    centralTrainLabelAligned = []
    combinedAlignedData = centralTestData
    for index, datasetName in enumerate(datasetList):
        if(datasetName == 'UCI'):
            centralTrainLabelAligned.append(centralTrainLabel[index])
            centralTestLabelAligned.append(centralTestLabel[index])
        elif(datasetName == 'RealWorld'):
            centralTrainLabelAligned.append(np.hstack([REALWORLD_CLIENT[labelIndex] for labelIndex in centralTrainLabel[index]]))
            centralTestLabelAligned.append(np.hstack([REALWORLD_CLIENT[labelIndex] for labelIndex in centralTestLabel[index]]))

        elif(datasetName == 'HHAR'):
            centralTrainLabelAligned.append(np.hstack([HHAR[labelIndex] for labelIndex in centralTrainLabel[index]]))
            centralTestLabelAligned.append(np.hstack([HHAR[labelIndex] for labelIndex in centralTestLabel[index]]))
        elif(datasetName == 'Motion_Sense'):
            centralTrainLabelAligned.append(np.hstack([Motion_Sense[labelIndex] for labelIndex in centralTrainLabel[index]]))
            centralTestLabelAligned.append(np.hstack([Motion_Sense[labelIndex] for labelIndex in centralTestLabel[index]]))
        else:
            centralTrainLabelAligned.append(np.hstack([SHL[labelIndex] for labelIndex in centralTrainLabel[index]]))
            centralTestLabelAligned.append(np.hstack([SHL[labelIndex] for labelIndex in centralTestLabel[index]]))
    centralTrainData = np.vstack((centralTrainData))
    centralTestData = np.vstack((centralTestData))
    centralTrainLabel = np.hstack((centralTrainLabelAligned))
    centralTestLabel = np.hstack((centralTestLabelAligned))
else:
    clientCount = utils.returnClientByDataset(dataSetName)
    datasetLoader = utils.loadDataset(dataSetName,clientCount,dataConfig,randomSeed,mainDir+'datasets/')
    centralTrainData = datasetLoader.centralTrainData
    centralTrainLabel = datasetLoader.centralTrainLabel

    centralTestData = datasetLoader.centralTestData
    centralTestLabel = datasetLoader.centralTestLabel

    clientOrientationTrain = datasetLoader.clientOrientationTrain
    clientOrientationTest = datasetLoader.clientOrientationTest
    orientationsNames = datasetLoader.orientationsNames

In [43]:
# If working on RealWorld or HHAR with specified position/device, we remove one and use it as the test set and combine the others for training
if(positionDevice != '' or dataSetName == 'UCI'):
    if(dataSetName == "RealWorld"):
        totalData = np.vstack((centralTrainData,centralTestData))
        totalLabel = np.hstack((centralTrainLabel,centralTestLabel))
        totalOrientation = np.hstack((np.hstack((clientOrientationTrain)), np.hstack((clientOrientationTest))))
        totalIndex = list(range(totalOrientation.shape[0]))
        testDataIndex = np.where(totalOrientation == orientationsNames.index(positionDevice))[0]
        trainDataIndex = np.delete(totalIndex,testDataIndex)

        centralTrainData = totalData[trainDataIndex]
        centralTestData = totalData[testDataIndex]

        centralTrainLabel = totalLabel[trainDataIndex]
        centralTestLabel = totalLabel[testDataIndex]
    elif(dataSetName == "HHAR"):
        totalData = np.vstack((centralTrainData,centralTestData))
        totalLabel = np.hstack((centralTrainLabel,centralTestLabel))
        totalOrientation = np.hstack((np.hstack((clientOrientationTrain)), np.hstack((clientOrientationTest))))
        totalIndex = list(range(totalOrientation.shape[0]))
        # 0 is for nexus
        testDataIndex = np.where(totalOrientation == orientationsNames.index(positionDevice))[0]
        trainDataIndex = np.delete(totalIndex,testDataIndex)

        centralTrainData = totalData[trainDataIndex]
        centralTestData = totalData[testDataIndex]

        centralTrainLabel = totalLabel[trainDataIndex]
        centralTestLabel = totalLabel[testDataIndex]
#         when using positions for evalaution, there is no test set , dev=test is the same
    centralDevData = centralTestData
    centralDevLabel = centralTestLabel
else:
#     using a 70 10 20 ratio
    centralTrainData, centralDevData, centralTrainLabel, centralDevLabel = train_test_split(centralTrainData, centralTrainLabel, test_size=0.125, random_state=randomSeed)

In [44]:
# Compute class weight
temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(centralTrainLabel),
                                                 y = centralTrainLabel.ravel())
class_weights = {j : temp_weights[j] for j in range(len(temp_weights))}

In [45]:
# One Hot of labels
centralTrainLabel = tf.one_hot(
    centralTrainLabel,
    activityCount,
    on_value=None,
    off_value=None,
    axis=None,
    dtype=None,
    name=None
)
centralTestLabel = tf.one_hot(
    centralTestLabel,
    activityCount,
    on_value=None,
    off_value=None,
    axis=None,
    dtype=None,
    name=None
)
centralDevLabel = tf.one_hot(
    centralDevLabel,
    activityCount,
    on_value=None,
    off_value=None,
    axis=None,
    dtype=None,
    name=None
)

In [46]:
optimizer = tf.keras.optimizers.Adam(learningRate)

if(architecture == "HART"):
    model_classifier = model.HART(input_shape,activityCount)
else:
    model_classifier = model.mobileHART_XS(input_shape,activityCount, convKernels = convKernels)
model_classifier.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
model_classifier.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 6)]     0           []                               
                                                                                                  
 sensor_patches (SensorPatches)  (None, 8, 192)      9408        ['input_1[0][0]']                
                                                                                                  
 patch_encoder (PatchEncoder)   (None, 8, 192)       1536        ['sensor_patches[0][0]']         
                                                                                                  
 normalizedInputs_0 (LayerNorma  (None, 8, 192)      384         ['patch_encoder[0][0]']          
 lization)                                                                                    

In [51]:
checkpoint_filepath = filepath+"bestValcheckpoint.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
    verbose=1,
)

start_time = time.time()
history = model_classifier.fit(
    x=centralTrainData,
    y=centralTrainLabel,
    validation_data = (centralDevData,centralDevLabel),
    batch_size=batch_size,
    epochs=localEpoch,
    verbose=showTrainVerbose,
    class_weight=class_weights,
    callbacks=[checkpoint_callback],
)
end_time = time.time() - start_time

model_classifier.save_weights(filepath + 'bestTrain.h5')
model_classifier.load_weights(checkpoint_filepath)
_, accuracy = model_classifier.evaluate(centralTestData, centralTestLabel)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")


Epoch 1/200


Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x79b31a16f520>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x79b31a16f520>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <bound method _BaseOptimizer._update_step_xla of <tensorflow.python.eager.polymorphic_function.tracing_compiler.TfMethodTarget object at 0x79b2556619f0>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <bound method _BaseOptimizer._update_step_xla of <tensorflow.python.eager.polymorphic_function.tracing_compiler.TfMethodTarget object at 0x79b2556619f0>>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
29/29 [==============================] - ETA: 0s - loss: 1.2065 - accuracy: 0.5896

Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x79b25494da20>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x79b25494da20>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

Epoch 1: val_accuracy improved from -inf to 0.71836, saving model to ./HART_Results/HART_16frameLength_16TimeStep_192ProjectionSize_0.0003LR/UCI/bestValcheckpoint.h5
29/29 [==============================] - 64s 130ms/step - loss: 1.2065 - accuracy: 0.5896 - val_loss: 1.0232 - val_accuracy: 0.7184
Epoch 2/200
29/29 [==============================] - ETA: 0s - loss: 0.8707 - accuracy: 0.7971
Epoch 2: val_accuracy improved from 0.71836 to 0.7

In [50]:
!pip install tensorflow --upgrade
!pip install keras --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 75.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
  